# 8 章 : データの可視化（Matplotlib, Seaborn を用いたグラフ作成）
----

## 8.1 解析環境のセットアップおよびデータの準備
### 8.1.1 可視化ライブラリ

- Matplotlib
  - プロット領域や座標軸等に対する細かい調整が容易にできる
  - 高度なグラフも描ける
- Seaborn
  - Matplotlib を補完する位置付け
  - 複雑なグラフを書くときに Matplotlib よりも簡単にコーディングできる

### 8.1.2 ライブラリのインストール

In [ ]:
# Matplotlib と Seaborn のインストール
!pip3 install matplotlib
!pip3 install seaborn

# Anaconda を使っている場合はこちら
# !conda install matplotlib
# !conda install seaborn

複数のヒストグラムを縦に並べて描いたり、ベン図を書いたりするのは、Matplotlib や Seaborn だけでは困難なので、JoyPy および matplotlib_venn ライブラリを使用する。

In [ ]:
# JoyPy と matplotlib_venn のインストール
!pip3 install joypy
!pip3 install matplotlib_venn

# Anaconda を使っている場合はこちら
# !conda install joypy
# !conda install matplotlib_venn

# 8.1.3 データセットの準備
本章の流れ
1. 擬似データを使用してグラフ作成関数の基本的な使い方を説明
2. 第 7 章で定量した遺伝子発現量行列を用いて、可視化の応用例を示す  
なお、発現量の低い遺伝子は、解析結果の偽陽性を増やす原因になるため、解析前に除去する必要がある。

In [ ]:
# 8-1 : データの読み込みと発現量の低い遺伝子の除去
import numpy as np
import pandas as pd

tpm = pd.read_table('count_tpm.tsv', index_col=0)
tpm = tpm.loc[tpm.means(axis=1) > 1.0, :]  # 平均発現量が 1.0TPM より大きいもののみ取り出す
tpm.head()  # データの確認

RNA-Seq データの定量で得られるリードカウントデータ、およびそれを正規化した TPM のばらつきは大きい  
→ わかりやすく可視化するためにデータの対数化

In [ ]:
# 8-2 : データの対数化
log_tpm = np.log10(tpm + 1.0)  # 真数が 0 になるのを防ぐために、1.0 を足しておく
log_tpm.head()

## 8.2 Matplotlib ライブラリの使い方
### 8.2.1 グラフのプロット領域

Matplotlib はグラフのプロット領域をいくつかのパーツに分けて、それぞれのパーツをクラスとして定義してある。
- Figure クラス : プロット領域全体を含む
- Axes クラス : 実際にグラフを描くサブプロット領域を含む

Figure クラスに対して、複数の Axis クラスのインスタンスを生成することで、1枚の画像に複数のグラフを描ける

<img src="matplotlib.jpeg" width=600>

### 8.2.2 グラフの作成方法
object-oriented インターフェースでグラフを書く手順
1. `matplotlib.pyplot.figure()` 関数で Figure クラスのインスタンスを呼び出す
2. Figure クラスの `add_subplot()` メソッドで Axes クラスのインスタンスを作成する
3. Axes クラスで定義されたサブプロット領域に、Axes クラスで定義された `scatter()` や `hist()` などの作図メソッドを使用してグラフを描く
4. 必要に応じて座標軸などのラベルをつける
5. Figure クラスの `show()` メソッドを使用してグラフを表示する

In [ ]:
# 8-3 : object-oriented インターフェースでのグラフの描画
import matplotlib.pyplot as plt

x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

fig = plt.figure()  # プロット領域を準備
ax = fig.add_subplot()  # グラフを描くためのサブプロット領域を用意
ax.scatter(x, y)  # 散布図を描く
# 軸ラベルを設定
ax.set_xlabel('X')
ax.set_ylabel('Y')
fig.show()  # ディスプレイに表示

state-based インターフェース
- MATLAB の使い方を踏襲した
- pyplot が現在操作中の Figure クラスや Axes クラスを自動的に識別して操作し、グラフを作成

In [ ]:
# 8-4 : state-based インターフェースでのグラフの描画
x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

plt.scatter(x, y)  # 自動的にプロット領域とサブプロット領域を用意したうえで散布図を描く

# 現在のサブプロット領域のグラフに対して軸ラベルを設定する
plt.xlabel('X')
plt.ylabel('Y')

plt.show()  # ディスプレイに表示

### 8.2.3 グラフの保存方法
`savefig()` メソッドに拡張子付きのファイル名を与えると、Figure クラスのインスタンス上で描かれたグラフが画像ファイルとして保存される
- PNG, PDF, SVG, TIFF などほとんどのフォーマットに対応している

画像のサイズは `plt.figure()` メソッドの引数 `figsize` を介して指定する
- `figsize=(width, height)`
- サイズの単位はインチ

解像度は引数 `dpi` を介して指定する

In [ ]:
# 8-5 : グラフを横 6.4 インチ、縦 4.8 インチ、解像度 100dpi の画像として PDF で書き出す
x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

fig - plt.figure(figsize=(6.4, 4.8), dpi=100)
ax = fig.add_subplot()
ax.scatter(x, y)
ax.set_xlabel('X')
ax.set_ylabel('Y')
fig.savefig('example_scatterplot.pdf')

state-based インターフェースを使用している場合、`plt.savefig()` メソッドでグラフを画像として保存できる。

### 8.2.4 基本グラフを描くメソッド
これらのメソッドは、x 座標および y 座標の配列（リストや pandas の Series でも可）を引数として受け取る  
その他に色、マーカーや線の形などを指定する引数も多数用意されている

<img src="8-2-4.jpeg" width=600>

### 8.2.5 座標軸や凡例を調整するメソッド
座標軸の表示範囲を設定したり、座標の目盛り位置を指定したり、座標軸のスケールを変更したりするためによく使われるメソッド

<img src="8-2-5.jpeg" width=600>

## 8.3 基本グラフ
この節では、ヒストグラム、ボックスプロット、散布図、線グラフ、棒グラフなどの基本的なグラフの作図方法を説明する。

### 8.3.1 ヒストグラム